In [1]:
import pandas as pd
import astropy.units as u
from astropy.coordinates import SkyCoord

from astroquery.vizier import Vizier
from filter_functions import filter_on_archival, filter_on_chandra

In [2]:
detections = pd.read_csv('detections_w20.txt', sep=' ', header=0)

for i, detection in detections.loc[2:2].iterrows():
    filter_on_chandra(detection)

b''


EmptyDataError: No columns to parse from file